In [1]:
import pandas as pd
#import sqlite3
#pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sb

titanic_data = pd.read_csv('train.csv', index_col = 'PassengerId')

SURVIVED_COL = 'Survived'
NAME_COL = 'Name'
SEX_COL = 'Sex'
CLASS_COL = 'Pclass'
AGE_COL = 'Age'
SIBLINGS_COL= 'SibSp'
FAMILY_COL = 'Parch'
TICKET_COL ='Ticket'
PRICE_COl = 'Fare'
CABIN_COL = 'Cabin'
PORT_COL = 'Embarked'

# Context

# Data quality assessment

# Data exploration

# Summary